In [1]:
import pandas as pd
data = pd.read_json("../doc2vec/data/all-notebooks-tokenized.json", orient='index')
import numpy as np
embeds2d = np.load("../doc2vec/data/notebooks-doc2vec-vectors-apr24.npy",allow_pickle=True)
embeds1d = []
for row in embeds2d:
    for vecs in row:
        embeds1d.append(vecs)

In [2]:
len(data[data.cell_type=="code"]
len(embeds1d)

856941

In [11]:
import argparse
import logging
import os
import pickle
import random
import torch
import json
import numpy as np
import pandas as pd
from model import BertModel, Generator
from torch.nn import CrossEntropyLoss, MSELoss
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler,TensorDataset
from gensim.models.doc2vec import Doc2Vec
import math
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm, trange
from pathlib import Path
import codecs
import json

from tokenize_code import tokenize_code

from utils import CellFeatures, InputFeatures, convert_examples_to_features, parseNotebook, get_notebook_list, get_embedding
from config import *

class RetrivalDB:
  def __init__(self):
    doc2vec_path = '../doc2vec/model'
    self.embed = np.asarray(embeds1d)
    self.df = data
    self.df = self.df[self.df.cell_type == 'code']
    # self.kernel_ids = np.load(doc2vec_path + "codebase_doc2vec_id.npy", allow_pickle=True)

  def getDoc(self, raw_idx):
    print("Raw idx is", raw_idx)
    if raw_idx < 0 or raw_idx >= self.embed.shape[0]:
      print("ERROR: out of index")
      return None
    
    return self.df.iloc[raw_idx]

  def find_sim(self, embed, topn=10):
    result = np.einsum("ij,ij->i",self.embed,embed)
    rank = np.argsort(-result)[:topn]
    doc_list = [self.getDoc(r) for r in rank]
    return doc_list

if __name__ == "__main__":
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  # TODO: change to doc2vec
  doc2vec_path = '../doc2vec/model'
  model = Doc2Vec.load(doc2vec_path + "/notebook-doc2vec-model-apr24.model")

  # gen = torch.load(doc2vec_path + "/best_gen.pt").to(device)
  gen = Generator(768, 768).to(device)
  gen = torch.load('gen_saved/best_gen.pt')
  gen.eval()
  db = RetrivalDB()
  
  while(True):
    print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
    input("Update the sample.py and press Enter to continue...")
    # TODO: reads ipynb
    input_file = './sample.ipynb'
    embed_list = []
    f = codecs.open(input_file, 'r')
    source = f.read()

    y = json.loads(source)
    for x in y['cells']:
        print(x)
        for x2 in x['source']:
            if x2[-1] != '\n':
                x2 = x2 + '\n'
            print("Input is", x2)
            embed_list.append(torch.Tensor(model.infer_vector(tokenize_code(x2,'code'))).to(device))

    print([e.shape for e in embed_list])

    predict_embed = gen.generate_embedding(embed_list)

    predict_embed = [embed.detach().cpu().numpy() for embed in predict_embed]
    
    doc_list = db.find_sim(predict_embed, topn=10)
    print(doc_list)
    file_path = '../doc2vec/data/sliced-notebooks-full-new'

    for col in doc_list:
#       print(col)
      print("############################")
      print(col.filename, col.competition)
      #print(col.source)


$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$


Update the sample.py and press Enter to continue... 


{'cell_type': 'code', 'execution_count': None, 'metadata': {}, 'outputs': [], 'source': []}
{'cell_type': 'code', 'execution_count': None, 'metadata': {}, 'outputs': [], 'source': ['\n']}
Input is 

{'cell_type': 'code', 'execution_count': None, 'metadata': {}, 'outputs': [], 'source': []}
[torch.Size([768])]
Raw idx is 38587
Raw idx is 128494
Raw idx is 472501
Raw idx is 541646
Raw idx is 139434
Raw idx is 135774
Raw idx is 546369
Raw idx is 599976
Raw idx is 540825
Raw idx is 534707
[cell_type                                                        code
source              d = run_length(mask.T.ravel())\n nmask = {}\n ...
filename                                                     18620591
competition                          severstal-steel-defect-detection
tokenized_source    [d, =, run_length, (, mask, ., T, ., ravel, (,...
Name: 52818, dtype: object, cell_type                                                        code
source              # # est. weights\n \n # def objective(w, 

Update the sample.py and press Enter to continue... 


{'cell_type': 'code', 'execution_count': None, 'metadata': {}, 'outputs': [], 'source': ['import pandas as pd\n', "df = pd.read_csv('1.csv')\n", 'df.drop()']}
Input is import pandas as pd

Input is df = pd.read_csv('1.csv')

Input is df.drop()

{'cell_type': 'code', 'execution_count': None, 'metadata': {}, 'outputs': [], 'source': ['df.head()\n', "index = (data['Region'] == 'Europe') + \\\n", "        (data.index == 'United States') + \\\n", "        (data.index == 'Canada') + \\\n", "        (data.index == 'Mexico')\n", 'selected = data[index]\n', '\n', 'print "By Total Gun Homicides"\n', 'sys.stdout.flush()\n', '\n', 'by_guns = selected.sort("Gun Homicides", ascending=False)\n', "#by_guns['Gun Homicides'].plot(kind='bar')\n", 'plot_percapita(by_guns, limit=25)\n', 'display_relevant(selected, limit=None)\n']}
Input is df.head()

Input is index = (data['Region'] == 'Europe') + \

Input is         (data.index == 'United States') + \

Input is         (data.index == 'Canada') + \

Input 

Update the sample.py and press Enter to continue... 


{'cell_type': 'code', 'execution_count': None, 'metadata': {}, 'outputs': [], 'source': ['from __future__ import division\n', 'import numpy as np\n', 'from scipy.special import factorial\n', 'import scipy.stats as stats\n', 'import pylab\n', 'import matplotlib.pyplot as plt\n', '%matplotlib inline\n', 'import seaborn as sns\n', 'sns.set_style("darkgrid")\n', 'import ipywidgets\n', 'from IPython.display import display\n', 'from matplotlib.font_manager import FontProperties\n', 'fontP = FontProperties()\n', "fontP.set_size('medium')\n", "%config InlineBackend.figure_format = 'svg'\n"]}
Input is from __future__ import division

Input is import numpy as np

Input is from scipy.special import factorial

Input is import scipy.stats as stats

Input is import pylab

Input is import matplotlib.pyplot as plt

Input is %matplotlib inline

Input is import seaborn as sns

Input is sns.set_style("darkgrid")

Input is import ipywidgets

Input is from IPython.display import display

Input is from matp

KeyboardInterrupt: Interrupted by user

In [4]:
gen = torch.load('gen_saved/best_gen.pt')
gen.eval()

Generator(
  (gru_unit): GRU(768, 768)
  (trans): Sequential(
    (0): Linear(in_features=768, out_features=768, bias=True)
    (1): Tanh()
    (2): Linear(in_features=768, out_features=768, bias=True)
  )
)

In [4]:
model = torch.load("gen_saved/doc2vec_best_gen.pt")